<a href="https://colab.research.google.com/github/Abhinavl3v3l/EVA3/blob/master/Assignment13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import time, math
from tqdm import tqdm_notebook as tqdm

import tensorflow as tf
import tensorflow.contrib.eager as tfe
import matplotlib.pyplot as plt
%matplotlib inline

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [0]:
from tensorflow.python.framework import ops
from tensorflow.python.ops import math_ops
from tensorflow.python.eager import context

def cyclic_learning_rate(global_step,
                         learning_rate=0.01,
                         max_lr=0.1,
                         step_size=20.,
                         gamma=0.99994,
                         mode='triangular',
                         name=None):
  """Applies cyclic learning rate (CLR).
     From the paper:
     Smith, Leslie N. "Cyclical learning
     rates for training neural networks." 2017.
     [https://arxiv.org/pdf/1506.01186.pdf]
      This method lets the learning rate cyclically
     vary between reasonable boundary values
     achieving improved classification accuracy and
     often in fewer iterations.
      This code varies the learning rate linearly between the
     minimum (learning_rate) and the maximum (max_lr).
      It returns the cyclic learning rate. It is computed as:
       ```python
       cycle = floor( 1 + global_step /
        ( 2 * step_size ) )
      x = abs( global_step / step_size – 2 * cycle + 1 )
      clr = learning_rate +
        ( max_lr – learning_rate ) * max( 0 , 1 - x )
       ```
      Polices:
        'triangular':
          Default, linearly increasing then linearly decreasing the
          learning rate at each cycle.
         'triangular2':
          The same as the triangular policy except the learning
          rate difference is cut in half at the end of each cycle.
          This means the learning rate difference drops after each cycle.
         'exp_range':
          The learning rate varies between the minimum and maximum
          boundaries and each boundary value declines by an exponential
          factor of: gamma^global_step.
       Example: 'triangular2' mode cyclic learning rate.
        '''python
        ...
        global_step = tf.Variable(0, trainable=False)
        optimizer = tf.train.AdamOptimizer(learning_rate=
          clr.cyclic_learning_rate(global_step=global_step, mode='triangular2'))
        train_op = optimizer.minimize(loss_op, global_step=global_step)
        ...
         with tf.Session() as sess:
            sess.run(init)
            for step in range(1, num_steps+1):
              assign_op = global_step.assign(step)
              sess.run(assign_op)
        ...
         '''
       Args:
        global_step: A scalar `int32` or `int64` `Tensor` or a Python number.
          Global step to use for the cyclic computation.  Must not be negative.
        learning_rate: A scalar `float32` or `float64` `Tensor` or a
        Python number.  The initial learning rate which is the lower bound
          of the cycle (default = 0.1).
        max_lr:  A scalar. The maximum learning rate boundary.
        step_size: A scalar. The number of iterations in half a cycle.
          The paper suggests step_size = 2-8 x training iterations in epoch.
        gamma: constant in 'exp_range' mode:
          gamma**(global_step)
        mode: one of {triangular, triangular2, exp_range}.
            Default 'triangular'.
            Values correspond to policies detailed above.
        name: String.  Optional name of the operation.  Defaults to
          'CyclicLearningRate'.
       Returns:
        A scalar `Tensor` of the same type as `learning_rate`.  The cyclic
        learning rate.
      Raises:
        ValueError: if `global_step` is not supplied.
      @compatibility(eager)
      When eager execution is enabled, this function returns
      a function which in turn returns the decayed learning
      rate Tensor. This can be useful for changing the learning
      rate value across different invocations of optimizer functions.
      @end_compatibility
  """
  if global_step is None:
    raise ValueError("global_step is required for cyclic_learning_rate.")
  with ops.name_scope(name, "CyclicLearningRate",
                      [learning_rate, global_step]) as name:
    learning_rate = ops.convert_to_tensor(learning_rate, name="learning_rate")
    dtype = learning_rate.dtype
    global_step = math_ops.cast(global_step, dtype)
    step_size = math_ops.cast(step_size, dtype)
    def cyclic_lr():
      """Helper to recompute learning rate; most helpful in eager-mode."""
      # computing: cycle = floor( 1 + global_step / ( 2 * step_size ) )
      double_step = math_ops.multiply(2., step_size)
      global_div_double_step = math_ops.divide(global_step, double_step)
      cycle = math_ops.floor(math_ops.add(1., global_div_double_step))
      # computing: x = abs( global_step / step_size – 2 * cycle + 1 )
      double_cycle = math_ops.multiply(2., cycle)
      global_div_step = math_ops.divide(global_step, step_size)
      tmp = math_ops.subtract(global_div_step, double_cycle)
      x = math_ops.abs(math_ops.add(1., tmp))
      # computing: clr = learning_rate + ( max_lr – learning_rate ) * max( 0, 1 - x )
      a1 = math_ops.maximum(0., math_ops.subtract(1., x))
      a2 = math_ops.subtract(max_lr, learning_rate)
      clr = math_ops.multiply(a1, a2)
      if mode == 'triangular2':
        clr = math_ops.divide(clr, math_ops.cast(math_ops.pow(2, math_ops.cast(
            cycle-1, tf.int32)), tf.float32))
      if mode == 'exp_range':
        clr = math_ops.multiply(math_ops.pow(gamma, global_step), clr)
      return math_ops.add(clr, learning_rate, name=name)
    if not context.executing_eagerly():
      cyclic_lr = cyclic_lr()
    return cyclic_lr

# New Section

In [0]:
# # lr_schedule = lambda t: np.interp([t], [0, (11250+1)//5, 11250], [0, 1, 0])[0]
# lr_schedule = lambda t: np.interp([t], [0, (EPOCHS+1)//5, EPOCHS], [0, LEARNING_RATE, 0])[0]
# lr_schedule(200/400)

# Assignment 13
1. Refer to your Assignment 12.
2. Replace whatever model you have there with the ResNet18 model as shown below.
Your model must look like Conv->B1->B2->B3->B4 and not individually called Convs. 
3. If not already using, then:
  - Use Batch Size 128
  - Use Normalization values of: (0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)
  - Random Crop of 32 with padding of 4px
  - Horizontal Flip (0.5)
  - Optimizer: SGD, Weight-Decay: 5e-4
  - NOT-OneCycleLR
  - Save model (to drive) after every 50 epochs or best model till now
4. Describe your blocks, and the stride strategy you have picked
5. Train for 300 Epochs
6. Assignment Target Accuracy is 90%, so exit gracefully if you reach 90% (you can target more, it can go till ~93%)
7. Assignment has hard deadline and any assignment submitted post deadline will not be accepted. 

In [0]:
tf.enable_eager_execution()

In [0]:
BATCH_SIZE = 128 #@param {type:"integer"}
MOMENTUM = 0.9 #@param {type:"number"}
LEARNING_RATE = 0.8 #@param {type:"number"}
WEIGHT_DECAY = 5e-4 #@param {type:"number"}
EPOCHS = 300 #@param {type:"integer"}
ITERATIONS = 11250 #@param {type:"integer"} 

# (60000/128) * 24 =  11250

https://mc.ai/tutorial-1-cifar10-with-google-colabs-free-gpu%E2%80%8A-%E2%80%8A92-5/

In [0]:
def init_pytorch(shape, dtype=tf.float32, partition_info=None):
  fan = np.prod(shape[:-1])
  bound = 1 / math.sqrt(fan)
  return tf.random.uniform(shape, minval=-bound, maxval=bound, dtype=dtype)

class ConvBN(tf.keras.Model):
  def __init__(self, c_out):
    super().__init__()
    self.conv = tf.keras.layers.Conv2D(filters=c_out, kernel_size=3, padding="SAME", kernel_initializer=init_pytorch, use_bias=False) ##  32*32*3 to 32*32*kernels.
    self.bn = tf.keras.layers.BatchNormalization(momentum=0.9, epsilon=1e-5)
    self.drop = tf.keras.layers.Dropout(0.05)

  def call(self, inputs):
    return tf.nn.relu(self.bn(self.drop(self.conv(inputs)))) 

class ResBlk(tf.keras.Model):
  def __init__(self, c_out, pool, res = False):
    super().__init__()
    self.conv_bn = ConvBN(c_out)
    self.pool = pool
    self.res = res
    if self.res:
      self.res1 = ConvBN(c_out)
      self.res2 = ConvBN(c_out)

  def call(self, inputs):
    h = self.pool(self.conv_bn(inputs))
    if self.res:
      h = h + self.res2(self.res1(h))
    return h

class DavidNet(tf.keras.Model):
  def __init__(self, c=64, weight=0.125):
    super().__init__()
    pool = tf.keras.layers.MaxPooling2D()
    self.init_conv_bn = ConvBN(c)
    self.blk1 = ResBlk(c*2, pool, res = True)             # kernels 64 * 2  = 128
    self.blk2 = ResBlk(c*4, pool)                         # kernels 64 * 4  = 256
    self.blk3 = ResBlk(c*8, pool, res = True)             # kernels 64 * 8  = 256
    self.blk4 = ResBlk(c*16, pool, res = True)            # kernels 64 * 16 = 1024
    self.pool = tf.keras.layers.GlobalMaxPool2D()
    self.linear = tf.keras.layers.Dense(10, kernel_initializer=init_pytorch, use_bias=False)
    self.weight = weight

  def call(self, x, y):
    h = self.pool(self.blk4(self.blk3(self.blk2(self.blk1(self.init_conv_bn(x)))))) #CN>B1>B2>B3>B4
    # h = self.pool(self.blk3(self.blk2(self.blk1(self.init_conv_bn(x)))))
    h = self.linear(h) * self.weight
    # h = h * self.weights
    ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=h, labels=y)
    loss = tf.reduce_sum(ce)
    correct = tf.reduce_sum(tf.cast(tf.math.equal(tf.argmax(h, axis = 1), y), tf.float32))
    return loss, correct

In [0]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
len_train, len_test = len(x_train), len(x_test)
y_train = y_train.astype('int64').reshape(len_train)
y_test = y_test.astype('int64').reshape(len_test)

train_mean = np.mean(x_train, axis=(0,1,2))
train_std = np.std(x_train, axis=(0,1,2))

normalize = lambda x: ((x - train_mean) / train_std).astype('float32') # todo: check here
pad4 = lambda x: np.pad(x, [(0, 0), (4, 4), (4, 4), (0, 0)], mode='reflect')


x_train = normalize(pad4(x_train))
x_test = normalize(x_test)

In [0]:
model = DavidNet()
batches_per_epoch = len_train//BATCH_SIZE + 1

# lr_schedule = lambda t: np.interp([t], [0, (EPOCHS+1)//5, EPOCHS], [0, LEARNING_RATE, 0])[0]

# lr_schedule = lambda t: np.interp([t], [0, (BATCH_SIZE+1)//10, BATCH_SIZE], [0, LEARNING_RATE, 0])[0]

# import matplotlib.pyplot as plt
# plt.plot([0, (BATCH_SIZE+1)//2, BATCH_SIZE], [0, LEARNING_RATE, 0], 'o')

rates = []

for i in range(0, 11250):
    x = cyclic_learning_rate(i, mode='triangular', gamma=1)
    rates.append(x())

# plt.xlabel('iterations (epochs)')
# plt.ylabel('learning rate')
# plt.plot(range(11250), rates)
rates=np.array(rates)
# rates = tf.convert_to_tensor(rates)
global_step = tf.train.get_or_create_global_step() # Number of batches seen.

# lr_func = lambda:(tf.convert_to_tensor(rates[i], dtype=tf.float32) for i in range(len(rates)))
# lr_func = lambda:(rates[i] for i in range(len(rates)))
def lr_func2():
  for i in range(len(rates)):
    return rates[i]


# lr_func = lambda: lr_schedule(global_step/batches_per_epoch)/BATCH_SIZE
opt = tf.train.MomentumOptimizer(lr_func2, momentum=MOMENTUM, use_nesterov=True)
data_aug = lambda x, y: (tf.image.random_flip_left_right(tf.random_crop(x, [32, 32, 3])), y)




In [16]:
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(data_aug).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  # print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t,'global step', global_step/1)
  print('epoch:', epoch+1, 'lr:', lr_func2(), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t,'global step', global_step/1)

epoch: 1 lr: 0.01 train loss: 0.9872787548828125 train acc: 0.65026 val loss: 0.9232954299926758 val acc: 0.6661 time: 55.11855745315552 global step tf.Tensor(2434.0, shape=(), dtype=float64)


epoch: 2 lr: 0.01 train loss: 0.8328553050231934 train acc: 0.70504 val loss: 0.9530941909790039 val acc: 0.6752 time: 110.69732618331909 global step tf.Tensor(2825.0, shape=(), dtype=float64)


epoch: 3 lr: 0.01 train loss: 0.7324910012054443 train acc: 0.7453 val loss: 0.7688423351287842 val acc: 0.7379 time: 165.65749883651733 global step tf.Tensor(3216.0, shape=(), dtype=float64)


epoch: 4 lr: 0.01 train loss: 0.6654797538757324 train acc: 0.76764 val loss: 0.6829610971450806 val acc: 0.7606 time: 220.91045546531677 global step tf.Tensor(3607.0, shape=(), dtype=float64)


epoch: 5 lr: 0.01 train loss: 0.6168128434753418 train acc: 0.78666 val loss: 0.8443127378463745 val acc: 0.7177 time: 276.157555103302 global step tf.Tensor(3998.0, shape=(), dtype=float64)


...

epoch: 296 lr: 0.01 train loss: 0.010056306077716872 train acc: 0.99668 val loss: 0.516917674255371 val acc: 0.9118 time: 16544.675695180893 global step tf.Tensor(117779.0, shape=(), dtype=float64)


epoch: 297 lr: 0.01 train loss: 0.009214000239921733 train acc: 0.99694 val loss: 0.5426835119247436 val acc: 0.911 time: 16601.30770969391 global step tf.Tensor(118170.0, shape=(), dtype=float64)


epoch: 298 lr: 0.01 train loss: 0.009476651106644422 train acc: 0.99708 val loss: 0.5290713563919067 val acc: 0.9106 time: 16657.408834934235 global step tf.Tensor(118561.0, shape=(), dtype=float64)


epoch: 299 lr: 0.01 train loss: 0.011871220123451202 train acc: 0.99636 val loss: 0.5466476169586182 val acc: 0.9073 time: 16714.02426958084 global step tf.Tensor(118952.0, shape=(), dtype=float64)


epoch: 300 lr: 0.01 train loss: 0.01147084481783677 train acc: 0.99622 val loss: 0.5360948832511901 val acc: 0.9104 time: 16770.472465753555 global step tf.Tensor(119343.0, shape=(), dtype=float64)
